In [4]:
# USAGE
# python train_liveness.py --dataset dataset --model liveness.model --le le.pickle

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from livenessnet import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os


In [5]:
data = []
labels = []

real_path = r'D:\ZaloAI\LIVENESS\dataset\real_img'
for filename in os.scandir(real_path):
    real_img = cv2.imread(filename.path)
    data.append(real_img)
    labels.append("real")

fake_path = r'D:\ZaloAI\LIVENESS\dataset\fake_img'
for filename in os.scandir(fake_path):
    fake_img = cv2.imread(filename.path)
    data.append(fake_img)
    labels.append("fake")

In [6]:

# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
# 	help="path to input dataset")
# ap.add_argument("-m", "--model", type=str, required=True,
# 	help="path to trained model")
# ap.add_argument("-l", "--le", type=str, required=True,
# 	help="path to label encoder")
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())

# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 50

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
# print("[INFO] loading images...")
# imagePaths = list(paths.list_images(args["dataset"]))
# data = []
# labels = []

# for imagePath in imagePaths:
# 	# extract the class label from the filename, load the image and
# 	# resize it to be a fixed 32x32 pixels, ignoring aspect ratio
# 	label = imagePath.split(os.path.sep)[-2]
# 	image = cv2.imread(imagePath)
# 	image = cv2.resize(image, (32, 32))

# 	# update the data and labels lists, respectively
# 	data.append(image)
# 	labels.append(label)

# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
	classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))

# save the network to disk
model_path = "liveness.model"
print("[INFO] serializing network to '{}'...".format(model_path))
model.save(model_path)

# save the label encoder to disk
le_path = "le.pickle"
f = open(le_path, "wb")
f.write(pickle.dumps(le))
f.close()



[INFO] compiling model...


c:\Users\gbhoa\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


[INFO] training network for 50 epochs...


c:\Users\gbhoa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1935: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
508/508 [==============================] - 13s 13ms/step - loss: 0.9311 - accuracy: 0.5505 - val_loss: 0.7286 - val_accuracy: 0.5873
Epoch 2/50
508/508 [==============================] - 6s 11ms/step - loss: 0.8537 - accuracy: 0.5731 - val_loss: 0.6803 - val_accuracy: 0.6308
Epoch 3/50
508/508 [==============================] - 5s 10ms/step - loss: 0.7860 - accuracy: 0.5911 - val_loss: 0.6703 - val_accuracy: 0.6404
Epoch 4/50
508/508 [==============================] - 5s 10ms/step - loss: 0.7513 - accuracy: 0.6103 - val_loss: 0.6701 - val_accuracy: 0.6308
Epoch 5/50
508/508 [==============================] - 5s 10ms/step - loss: 0.7479 - accuracy: 0.6081 - val_loss: 0.6441 - val_accuracy: 0.6603
Epoch 6/50
508/508 [==============================] - 5s 10ms/step - loss: 0.7025 - accuracy: 0.6359 - val_loss: 0.5979 - val_accuracy: 0.6971
Epoch 7/50
508/508 [==============================] - 5s 10ms/step - loss: 0.6889 - accuracy: 0.6457 - val_loss: 0.6223 - val_accuracy: 0.698

In [23]:
# plot the training loss and accuracy
plot_path = "plot.png"
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)

In [21]:
print(H.history)

{'loss': [0.921705424785614, 0.826959490776062, 0.8148155808448792, 0.7609246969223022, 0.7262651920318604, 0.7257144451141357, 0.6929773688316345, 0.6642053127288818, 0.6601290106773376, 0.6517049670219421, 0.6397360563278198, 0.6391095519065857, 0.6222884058952332, 0.6199017763137817, 0.6010035872459412, 0.5980288982391357, 0.5832599997520447, 0.5869350433349609, 0.5858795642852783, 0.5698794722557068, 0.561039388179779, 0.5669911503791809, 0.5632133483886719, 0.5600284337997437, 0.5557820796966553, 0.5446659326553345, 0.5476539731025696, 0.5314794778823853, 0.5394994616508484, 0.5442585945129395, 0.5313345193862915, 0.5268172025680542, 0.5231662392616272, 0.5131305456161499, 0.517967164516449, 0.5100886821746826, 0.5207077264785767, 0.519129753112793, 0.5101861357688904, 0.5068457722663879, 0.5088955163955688, 0.5117530822753906, 0.5130264163017273, 0.49954378604888916, 0.5118096470832825, 0.4990816116333008, 0.5091240406036377, 0.5056278705596924, 0.5073855519294739, 0.497554391622